In [1]:
"""
Members : 

- Ekane Emile
- LOUOKDOM FOKAM Neal
- SIEWE DAHE William
- NZEKET Aïcha

"""

/usr/local/lib/python3.9/site-packages/matplotlib/__init__.py:152: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if LooseVersion(module.__version__) < minver:
/usr/local/lib/python3.9/site-packages/setuptools/_distutils/version.py:351: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  other = LooseVersion(other)
/usr/local/lib/python3.9/site-packages/matplotlib/__init__.py:152: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if LooseVersion(module.__version__) < minver:
/usr/local/lib/python3.9/site-packages/setuptools/_distutils/version.py:351: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  other = LooseVersion(other)
/usr/local/lib/python3.9/site-packages/matplotlib/__init__.py:152: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if LooseVersion(module

In [70]:
import pandas as pd
import re 
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
from nltk import word_tokenize, pos_tag
from gensim.models.ldamulticore import LdaMulticore
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
import glob
import gensim.corpora as corpora
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis
import json
import string
import random 
pyLDAvis.enable_notebook()
%matplotlib inline

In [2]:
df = pd.read_csv('question_responce.csv')
df = df[["responce", "question"]]

In [3]:
def cleaningfunc (column):
    cleaned_data = []
    sentence = []
    data = []

    for line in column:
        cleaned_data.append(re.sub(r'[^\w\s]', ' ',re.sub(r"(?:\@|https?\://)\S+", "", re.sub("#[A-Za-z0-9_]+","", line)).lower()))

    for line in cleaned_data:
        for word in line.split():
            if word not in stopwords.words("english"):
                sentence.append(word)
        data.append((" ").join(sentence))
        sentence = []
    cleaned_data = data
    
    return cleaned_data

In [4]:
def tokenizingfunc(data_cleaned):
    data_tokenized = [word_tokenize(line) for line in data_cleaned]
    data_tokenized = [word for word in data_tokenized if word not in stopwords.words("english")]
    return data_tokenized

def tokenizingfuncquestion(text):
    cleaned_data = []
    data = re.sub(r'[^\w\s]', ' ',re.sub(r"(?:\@|https?\://)\S+", "", re.sub("#[A-Za-z0-9_]+","", text)).lower())
    
    for word in data.split():
        if word not in stopwords.words("english"):
                cleaned_data.append(word)
    
    return cleaned_data

In [5]:
def get_wordnet_pos(treebank_tag):
    if treebank_tag.startswith('J'):
        return wordnet.ADJ
    elif treebank_tag.startswith('V'):
        return wordnet.VERB
    elif treebank_tag.startswith('N'):
        return wordnet.NOUN
    elif treebank_tag.startswith('R'):
        return wordnet.ADV
    return ''

def lemmatizingfunc(data_tokenized):
    lemmatizer = WordNetLemmatizer()

    data_lemmatized = []
    line_lemmatized = []

    for line in data_tokenized:
        for i, word in enumerate(line):
            pos = get_wordnet_pos(pos_tag([word])[0][1])
            if pos != '':
                line_lemmatized.append(lemmatizer.lemmatize(word, pos))
            else:
                line_lemmatized.append(word)
        data_lemmatized.append(line_lemmatized)
        line_lemmatized = []
    
    return data_lemmatized

def lemmatizingfuncquestion(data_tokenized):
    lemmatizer = WordNetLemmatizer()

    data_lemmatized = []

    for word in data_tokenized:
        pos = get_wordnet_pos(pos_tag([word])[0][1])
        if pos != '':
            data_lemmatized.append(lemmatizer.lemmatize(word, pos))
        else:
            data_lemmatized.append(word)
    
    return data_lemmatized
            

In [6]:
def preprocessingquestion(text):
    return lemmatizingfuncquestion(tokenizingfuncquestion(text))

In [7]:
question_cleaned = cleaningfunc(df["question"])
question_tokenized = tokenizingfunc(question_cleaned)
question_lemmatized = lemmatizingfunc(question_tokenized)

response_cleaned = cleaningfunc(df["responce"])
response_tokenized = tokenizingfunc(response_cleaned)
response_lemmatized = lemmatizingfunc(response_tokenized)

In [8]:
def doc2vecfunc():
    documents = glob.glob("corpus/*")
    documents = [TaggedDocument(doc, [i]) for i, doc in enumerate(df['responce'])]
    model = Doc2Vec(vector_size=10, alpha=0.025,min_alpha=0.00025,min_count=10,dm=0, epochs=150)
    model.build_vocab(documents)
    model.train(documents, total_examples=model.corpus_count, epochs=model.epochs)
    return model

In [9]:
def ldafunc(data_lemmatized):
    id2word = corpora.Dictionary(data_lemmatized)
    
    # Create Corpus
    texts = data_lemmatized

    # Term Document Frequency
    corpus = [id2word.doc2bow(text) for text in texts]

    lda = LdaMulticore(corpus = corpus, id2word=id2word, num_topics=25, passes=5)
    
    pyLDAvis.enable_notebook()
    vis = gensimvis.prepare(lda, corpus, id2word)

    return lda,corpus,vis

In [10]:
model = doc2vecfunc()
lda,corpus,vis = ldafunc(question_lemmatized)

/usr/local/lib/python3.9/site-packages/pyLDAvis/_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  default_term_info = default_term_info.sort_values(
/usr/local/lib/python3.9/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/usr/local/lib/python3.9/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/usr/local/lib/python3.9/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/usr/local/lib/python3.9/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is dep

In [11]:
for idx, topic in lda.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx, topic))

Topic: 0 
Words: 0.016*"seat" + 0.012*"see" + 0.010*"much" + 0.010*"flight" + 0.010*"go" + 0.009*"take" + 0.009*"give" + 0.009*"get" + 0.008*"hour" + 0.007*"say"
Topic: 1 
Words: 0.050*"flight" + 0.040*"get" + 0.011*"service" + 0.010*"time" + 0.008*"know" + 0.008*"thanks" + 0.008*"miss" + 0.008*"home" + 0.007*"delay" + 0.007*"plane"
Topic: 2 
Words: 0.040*"flight" + 0.013*"still" + 0.012*"board" + 0.011*"get" + 0.011*"us" + 0.010*"gate" + 0.009*"hey" + 0.009*"back" + 0.008*"hour" + 0.008*"fly"
Topic: 3 
Words: 0.042*"flight" + 0.013*"bad" + 0.011*"morning" + 0.010*"cancel" + 0.010*"know" + 0.010*"delayed" + 0.010*"get" + 0.009*"make" + 0.009*"would" + 0.008*"try"
Topic: 4 
Words: 0.042*"flight" + 0.016*"get" + 0.014*"refund" + 0.012*"fly" + 0.012*"bag" + 0.010*"first" + 0.010*"never" + 0.009*"book" + 0.009*"gt" + 0.008*"hour"
Topic: 5 
Words: 0.042*"flight" + 0.015*"take" + 0.012*"make" + 0.011*"another" + 0.010*"amp" + 0.010*"get" + 0.008*"thanks" + 0.008*"time" + 0.007*"plane" + 0.00

In [33]:
ourData = {"intents":[
    {
        "tag" : "Thanks",
        "patterns" : ["thanks", "thanksgiving", "thank", "happy", "great", "beautiful", "nice"],
        "responses" : ["I am happy to hear that", "We will continue to work hard to maintain your satifaction"]
    },
    {
        "tag" : "Delay",
        "patterns" : ["delay", "late", "wait"],
        "responses" : ["We are really sorry to hear that", "We sincerely apologize about this delay"]
    },
    {
        "tag" : "Cancel",
        "patterns" : ["cancel"],
        "responses" : ["We're sorry about that inconvenient situation. Rebook another flight"]
    },
    {
        "tag" : "Guidance",
        "patterns" : ["guide"],
        "responses" : ["We'll see what we could do", "We'll send you someone as soon as possible"]
    },
    {
        "tag" : "Goodbye",
        "patterns" : ["bye", "goodbye"],
        "responses" : ["You're welcome, goodbye and have a nice stay"]
    },
    {
        "tag" : "Please",
        "patterns" : ["please"],
        "responses" : ["I will do the best I can"]
    },
    {
        "tag" : "Disappointment",
        "patterns" : ["disappoint", "upset"],
        "responses" : ["We are really sorry to hear that, we shall do better next time"]
    },
    {
        "tag" : "Refund",
        "patterns" : ["refund"],
        "responses" : ["We are really sorry, our team will study your request and then, give you a feedback"]
    },
    {
        "tag" : "Help",
        "patterns" : ["help"],
        "responses" : ["I'll do my best for that", "We'll send you someone asap"]
    }
]}

In [13]:
vis

PreparedData(topic_coordinates=              x         y  topics  cluster      Freq
topic                                               
9     -0.034582 -0.042415       1        1  9.171831
1     -0.055862 -0.019822       2        1  6.160946
8     -0.052191 -0.042620       3        1  4.914160
12     0.012101  0.005776       4        1  4.809520
4     -0.058541 -0.004044       5        1  4.794924
2     -0.053670 -0.069877       6        1  4.747109
5     -0.046496 -0.024516       7        1  4.307896
22     0.021441  0.031308       8        1  4.151056
6      0.054260  0.104445       9        1  3.950858
15    -0.004749 -0.058388      10        1  3.915003
24    -0.042944  0.078315      11        1  3.803471
14    -0.046085  0.061799      12        1  3.755559
16     0.022253 -0.021356      13        1  3.648212
10    -0.007497  0.083750      14        1  3.613291
18    -0.031748  0.019402      15        1  3.516282
7      0.058730  0.061871      16        1  3.492287
11     0.071111 -0.052152      17        1  3.477342
23     0.092407 -0.012365      18        1  3.149441
19     0.062139 -0.008960      19        1  3.089691
13     0.032990 -0.067470      20        1  3.081519
21    -0.072656  0.003899      21        1  3.034419
17    -0.023444 -0.050932      22        1  2.995660
20     0.066678 -0.078305      23        1  2.855507
3     -0.087661  0.061893      24        1  2.788122
0      0.124017  0.040763      25        1  2.775894, topic_info=        Term        Freq       Total Category  logprob  loglift
67    ticket   62.000000   62.000000  Default  30.0000  30.0000
117    check   83.000000   83.000000  Default  29.0000  29.0000
299   thanks  114.000000  114.000000  Default  28.0000  28.0000
10      gate  114.000000  114.000000  Default  27.0000  27.0000
30     great   63.000000   63.000000  Default  26.0000  26.0000
..       ...         ...         ...      ...      ...      ...
439    plane    2.674473   95.953867  Topic25  -5.2972   0.0041
56     still    2.483372   68.534901  Topic25  -5.3713   0.2665
206  delayed    2.468115   55.831548  Topic25  -5.3775   0.4653
30     great    2.423452   63.640277  Topic25  -5.3957   0.3161
114      amp    2.415252   66.192294  Topic25  -5.3991   0.2734

[1909 rows x 6 columns], token_table=      Topic      Freq    Term
term                         
252       2  0.237541     000
252      14  0.237541     000
252      17  0.237541     000
2148      8  0.600474      01
829       9  0.594119    05am
...     ...       ...     ...
844      20  0.051214     yet
844      22  0.051214     yet
3545     12  0.600336  yogurt
1581      1  0.573702   youre
2940     14  0.607074   youth

[5354 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[10, 2, 9, 13, 5, 3, 6, 23, 7, 16, 25, 15, 17, 11, 19, 8, 12, 24, 20, 14, 22, 18, 21, 4, 1])

In [14]:
for index, score in sorted(lda[corpus[65]], key=lambda tup: -1*tup[1]):
    print("\nScore: {}\t \nTopic: {}".format(score, lda.print_topic(index, 10)))


Score: 0.9039914011955261	 
Topic: 0.019*"get" + 0.019*"flight" + 0.009*"bag" + 0.009*"way" + 0.009*"go" + 0.007*"aa" + 0.007*"blah" + 0.007*"well" + 0.007*"call" + 0.007*"ask"


In [15]:
topics = []
for i in range(len(df["responce"])):
    topics.append(sorted(lda[corpus[i]], key = lambda x : -1*x[1])[0][0])

df['topics'] = topics

In [96]:
id = 154
question = lemmatizingfuncquestion(tokenizingfuncquestion(df["question"].loc[id]))

In [17]:
id2word = corpora.Dictionary([question])
corpus = [id2word.doc2bow(question)]

In [18]:
model.dv.most_similar(model.infer_vector(question))

[(937, 0.6438348889350891),
 (1572, 0.6359788775444031),
 (1177, 0.6345400810241699),
 (351, 0.6335066556930542),
 (1546, 0.6232224702835083),
 (1223, 0.6206983923912048),
 (136, 0.6050586700439453),
 (1578, 0.6031753420829773),
 (1218, 0.6020288467407227),
 (360, 0.5948842167854309)]

In [97]:
print("question : " + df["question"].loc[id] + "\n")
print("response : " + df["responce"].loc[id])

question : @AmericanAir I can understand delays for various reasons but a 6:25am departure where your #captain didn't get a wake up call? #MakeItRight

response : @122510 Our apologies for the late start. We'll get our crew complete and get you going soon.


In [20]:
df["responce"].loc[model.dv.most_similar(model.infer_vector(question))[0][0]]

'@150829 Please check-in for the flights you can and then the agents at the airport will print out the remaining boarding passes for the others.'

In [21]:
"""mask = df['topics'] == lda[corpus[0]][0][0]
df = df[mask]
df["responce"].loc[model.dv.most_similar(model.infer_vector(question))[0][0]]
"""

'mask = df[\'topics\'] == lda[corpus[0]][0][0]\ndf = df[mask]\ndf["responce"].loc[model.dv.most_similar(model.infer_vector(question))[0][0]]\n'

In [22]:
#bot = ChatBot(name="AiBot", read_only=False, logic_adaptaters = ["chatterbot.logic.BestMatch"])

In [23]:
#corpus_trainer = ChatterBotCorpusTrainer(bot)
#corpus_trainer.train('chatterbot.corpus.english')




In [24]:
#print(bot.get_response("hi"))

In [ ]:
"""lm = WordNetLemmatizer() #for getting words
# lists
ourClasses = []
newWords = []
documentX = []
documentY = []
# Each intent is tokenized into words and the patterns and their associated tags are added to their respective lists.
for intent in ourData["intents"]:
    for pattern in intent["patterns"]:
        ourTokens = word_tokenize(pattern)
        newWords.extend(ourTokens)
        documentX.append(pattern)
        documentY.append(intent["tag"])
    
    
    if intent["tag"] not in ourClasses:# add unexisting tags to their respective classes
        ourClasses.append(intent["tag"])

newWords = [lm.lemmatize(word.lower()) for word in newWords if word not in string.punctuation] # set words to lowercase if not in punctuation
newWords = sorted(set(newWords))# sorting words
ourClasses = sorted(set(ourClasses))# sorting classes"""

In [98]:
def getRes(text, fJson):
  listOfIntents = fJson["intents"]
  for w in text:
    for i in listOfIntents: 
      for el in i["patterns"]:
        if el == w:
          ourResult = random.choice(i["responses"])
          print(ourResult)
          return ourResult

getRes(question, ourData)

We sincerely apologize about this delay


'We sincerely apologize about this delay'

In [66]:
print(getRes(question, ourData))
#print(ourResult)

None
